In [2]:
import cv2


In [3]:
#connecting to capture device 
import cv2
cap = cv2.VideoCapture(1) #selecting source, in this case a webcam 

#tracker = cv2.legacy.TrackerMOSSE_create()

def capture_webcam(cap):
    '''
    This function is just to stream the video from a webcam 
    '''
    if not cap.isOpened(): #error handling 
        return "Error opening the video."
    while cap.isOpened():   #when there is video, capture it and stream it 
        timer = cv2.getTickCount()
        success, img = cap.read() #unpacking and then the frame

        fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) #calculating the fps value 
        cv2.putText(img, str(int(fps)), (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        #calculate fps 
        if success:
            #gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            cv2.imshow("Tracking", img)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                break
        else:
            break

In [1]:
'''
This section is a working example
It captures and streams the images from a the webcam as long as there is an image to stream 
'''

import cv2

cap = cv2.VideoCapture(1) #selecting source, in this case a webcam 

tracker = cv2.legacy.TrackerMOSSE_create()
success, img = cap.read()
bbox = cv2.selectROI(img, False)
print(bbox)
tracker.init(img,bbox)

def drawBox(img, bbox):
    ''' 
    Function to draw a bounding box
    Input parameters are the image 
    and bbox are the x and y coordinates and w and h 
    '''
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(img, p1, p2, (255,0,0), 2, 1)
    cv2.putText(img, 
                'Tracking', 
                (75,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,255,0), 
                2)
    cv2.putText(img, 
                str(bbox), 
                (75, 25), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,255,0), 
                2)

if not cap.isOpened(): #error handling 
    print("Error opening the video.")
while cap.isOpened():   #when there is video, capture it and stream it 
    timer = cv2.getTickCount()
    success, img = cap.read() #unpacking and then the frame  
    
    #bbox = cv2.selectROI("Tracking", img, False)
    
    ret, bbox = tracker.update(img)
    
    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) #calculating the fps value 
    
    cv2.putText(img, 
                str(int(fps)), 
                (75,75), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,0,255), 
                2)
    if ret:
        drawBox(img, bbox)
    else:
        cv2.putText(img, 
                'Failure detected', 
                (75,50), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,0,255), 
                2)
    if success:
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imshow("Tracking", img)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
    else:
        break

(398, 260, 239, 160)


In [4]:
capture_webcam(cap)

In [1]:
''' 
This is the 2nd version 
I attempted to use mouse events to draw the box 
'''

import cv2

cap = cv2.VideoCapture(1)
p1, p2 = None, None
state = 0
tracker = cv2.legacy.TrackerMOSSE_create()
def drawBox(img, bbox):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(img, p1, p2, (255,0,0), 2, 1)
    cv2.putText(img, 
                'Tracking', 
                (75,100), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,255,0), 
                2)
    cv2.setMouseCallback('Tracking', on_mouse)

def on_mouse(event, x, y, flags, userdata):
    global state, p1, p2, bbox
    #left click
    if event == cv2.EVENT_LBUTTONUP:
        #select first point
        if state == 0:
            p1 = (x, y)
            state += 1
        #select second point
        elif state == 1:
            p2 = (x, y)
            state += 1
            
    #right click (erase current ROI)
    if event == cv2.EVENT_RBUTTONUP:
        p1, p2 = None, None
        state = 0

    
if not cap.isOpened(): #error handling 
    print("Error opening the video.")
while cap.isOpened():   #when there is video, capture it and stream it 
    timer = cv2.getTickCount()
    success, img = cap.read() #unpacking and then the frame
    
    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) #calculating the fps value 
    cv2.putText(img, str(int(fps)), (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    #calculate fps
    #bbox = (p1 + p2)
    ret, bbox = tracker.update(img)
    
    if ret:
        event = None
        drawBox(img, bbox)
        
    else:
        cv2.putText(img, 
        "Failure", 
        (75,75), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        0.7, 
        (0,0,255), 
        2)

    if success:
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if state > 1:
            # bbox = (p1 + p2)
            x, y = p1
            w, h = ((p2[0] - p1[0]), (p2[1] - p1[1]))
            bbox = (x, y, w, h)
            #cv2.rectangle(img, p1, p2, (255,0,0), 2, 1)  
            tracker.init(img,bbox)       

        cv2.imshow("Tracking", img)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
    else:
        break

    cv2.setMouseCallback('Tracking', on_mouse)

In [6]:
'''
3rd version
On top of the code above
I am attempting to delete the current tracker and bbox and draw a new one, without stopping the stream of video 

WIP: Without success yet
'''

import cv2
cap = cv2.VideoCapture(1)
p1, p2 = None, None
state = 0
tracker = cv2.legacy.TrackerMOSSE_create()
def drawBox(img, bbox):
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(img, p1, p2, (255,0,0), 2, 1)
    cv2.putText(img, 
                'Tracking', 
                (75,100), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,255,0), 
                2)
    cv2.putText(img, 
                str(bbox), 
                (75,75), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, 
                (0,255,0), 
                2)
    cv2.setMouseCallback('Tracking', on_mouse)

def on_mouse(event, x, y, flags, userdata):
    global state, p1, p2, bbox
    #left click
    if event == cv2.EVENT_LBUTTONUP:
        #select first point
        if state == 0:
            p1 = (x, y)
            state += 1
        #select second point
        elif state == 1:
            p2 = (x, y)
            state += 1
            
    #right click (erase current ROI)
    if event == cv2.EVENT_RBUTTONUP:
        p1, p2 = None, None
        state = 0
        bbox = 0

if not cap.isOpened(): #error handling 
    print("Error opening the video.")
while cap.isOpened():   #when there is video, capture it and stream it 
    timer = cv2.getTickCount()
    success, img = cap.read() #unpacking and then the frame
    
    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer) #calculating the fps value 
    cv2.putText(img, str(int(fps)), (75,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    #calculate fps
    #bbox = (p1 + p2)
    ret, bbox = tracker.update(img)
    
    if ret:
        event = None
        drawBox(img, bbox)
        
    else:
        cv2.putText(img, 
        "Failure", 
        (75,75), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        0.7, 
        (0,0,255), 
        2)

    if success:
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if state > 1:
            # bbox = (p1 + p2)
            x, y = p1
            w, h = ((p2[0] - p1[0]), (p2[1] - p1[1]))
            bbox = (x, y, w, h)
            #cv2.rectangle(img, p1, p2, (255,0,0), 2, 1)  
            tracker.init(img,bbox)

        cv2.imshow("Tracking", img)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
    else:
        break

    cv2.setMouseCallback('Tracking', on_mouse)

In [ ]:
#edit # 1 just testing